In [1]:
import pipeline.pipeline as pipeline
import db.knowhere_db as kdb
import time

In [ ]:
fName = r'C:/Users/William/Documents/DataScience/Projects/Knowhere/data/aggregated_emil_sensor2.csv'

In [ ]:
user_df = pipeline.iphone(user_id='58c5b91d71a9e82f33ad1668', file_with_path=fName)

In [ ]:
user_df.to_dict(orient='records')

In [ ]:
writer = kdb.Writer('knowhere')
writer.write_dataframe_to_collection(df=user_df, collection='iphone_test')

In [2]:
reader = kdb.Reader('knowhere')
rdf = reader.get_dataframe(collection='iphone_test')
rdf[0:10]

,_id,data,sensor,timestamp,user_id
0,58c80246ea2bfa24d01dc805,"{u'Latitude': u'40.7530392755199', u'Vertical ...",GPS,2017-03-08 10:38:48,58c5b91d71a9e82f33ad1668
1,58c80246ea2bfa24d01dc806,"{u'y': u'0.0504199266433716', u'x': u'-0.00513...",Acceleration,2017-03-08 10:38:49,58c5b91d71a9e82f33ad1668
2,58c80246ea2bfa24d01dc807,"{u'Pressure': u'101.2276611328125', u'Relative...",Altimeter (Barometer),2017-03-08 10:38:49,58c5b91d71a9e82f33ad1668
3,58c80246ea2bfa24d01dc808,"{u'Yaw': u'-0.422258717009882', u'Quarternion ...",Attitude,2017-03-08 10:38:49,58c5b91d71a9e82f33ad1668
4,58c80246ea2bfa24d01dc809,"{u'State': u'Discharging', u'Level': u'0.55'}",Battery,2017-03-08 10:38:49,58c5b91d71a9e82f33ad1668
5,58c80246ea2bfa24d01dc80a,{u'Status': u'Off'},Bluetooth,2017-03-08 10:38:49,58c5b91d71a9e82f33ad1668
6,58c80246ea2bfa24d01dc80b,"{u'Radio Access Technology': u'4G (LTE)', u'Ca...",Cell Radio,2017-03-08 10:38:49,58c5b91d71a9e82f33ad1668
7,58c80246ea2bfa24d01dc80c,"{u'Latitude': u'40.7530392777554', u'Vertical ...",GPS,2017-03-08 10:38:49,58c5b91d71a9e82f33ad1668
8,58c80246ea2bfa24d01dc80d,"{u'y': u'-0.226155400276184', u'x': u'0.081479...",Gravity,2017-03-08 10:38:49,58c5b91d71a9e82f33ad1668
9,58c80246ea2bfa24d01dc80e,"{u'y': u'-1.38406085968018', u'x': u'-0.787979...",Gyrometer,2017-03-08 10:38:49,58c5b91d71a9e82f33ad1668


In [ ]:
import pymongo
import ConfigParser
from pandas import DataFrame

cntr = 0

for entry in reader.filter_collection('iphone_test'):
    row = {'timestamp': entry['timestamp'], 'user_id': entry['user_id'], 'sensor': entry['sensor']}
    if cntr > 7:
        break
    for data_name, data_raw in entry['data'].iteritems():
        if cntr>7:
            break
        row['data_name'] = data_name
        row['data_raw'] = data_raw
        print row
        cntr += 1

In [ ]:
import pandas as pd

rdf2 = rdf.copy()
rdf2[['data_name', 'data_raw']] = rdf2.data.apply(lambda data: pd.Series([tuple(data.keys()), tuple(data.values())]))
rdf2.drop(['data'], axis=1, inplace=True)
rdf2[:3]
rdf2.groupby(['_id', 'sensor', 'user_id']).agg(pd.Series)